In [ ]:
# !pip install googleapiclient

In [10]:
import pandas as pd
from googleapiclient.discovery import build
 
 
api_key = 'youtube_api'
video_id = 'video_id'
 
comments = []

api_obj = build('youtube', 'v3', developerKey=api_key)
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()

while response:
    if 'items' in response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount'], 'Top Level'])

            if 'replies' in item:
                for reply_item in item['replies']['comments']:
                    reply = reply_item['snippet']
                    comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount'], 'Reply'])

    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break

df = pd.DataFrame(comments, columns=['comment', 'author', 'date', 'num_likes', 'comment_type'])
df.to_excel('results.xlsx', index=None)